In [35]:
import numpy as np 
import pandas as pd

import bs4
import requests

### Popular products

In [5]:
def get_items_link(start_link) : 
    req = requests.get(start_link)
    parser = bs4.BeautifulSoup(req.text, 'lxml')

    items_coments_link = []
    for link in parser.select('div.g-i-tile-i-title > a'):
        items_coments_link.append(link['href'] + 'comments/page=1;sort=helpful/')
    
    return items_coments_link

In [6]:
items_coments_link = get_items_link('https://rozetka.com.ua/mobile-phones/c80003/')

In [7]:
print(len(items_coments_link))
items_coments_link[:5]

32


['https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=1;sort=helpful/',
 'https://rozetka.com.ua/nokia_5_ds_blue/p16104620/comments/page=1;sort=helpful/',
 'https://rozetka.com.ua/viaan_v182_black/p23575076/comments/page=1;sort=helpful/',
 'https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=1;sort=helpful/',
 'https://rozetka.com.ua/nokia_216_dual_sim_grey/p11684285/comments/page=1;sort=helpful/']

In [13]:
def get_last_coment_page(parser) :
    '''return last page in coment and parser'''
    try :
        last_page = int([tag.text for tag in parser.select('li.paginator-catalog-l-i > span')][-1])
    except IndexError :
        last_page = 1
    return last_page

def evaluate_stars_count(starts) :
    starts = np.array(starts)
    
    for i in range(5,-1,-1) :
        print(i,np.sum(starts == i))

In [14]:
reviws_sample = []
reviws_stars = []
page_count = []

In [15]:
def parse_link(items_coments_link,reviws_sample,reviws_stars,page_count) :
    
    for indx,coment_link in enumerate(items_coments_link) :

        print(indx,'start prosess page',coment_link)
        req = requests.get(coment_link)
        print(req)
        parser = bs4.BeautifulSoup(req.text,'lxml')

        if (parser.find('li',attrs={'class' : 'm-tabs-i', 'name' : 'comments'}).a.text != "\nОставить отзыв") :

            last_page= get_last_coment_page(parser)
            page_count.append(last_page)

            for tag in parser.select('div.pp-review-inner'): #> > div.pp-review-i-b

                reviws = ' '.join([div.text for div in tag.find_all('div', attrs = {'class' : 'pp-review-text-i'})])
                #print(reviws)

                starts = tag.find('span', attrs = {'class' : 'g-rating-stars-i'})
                #print(starts)

                if reviws != None and starts != None :
                    reviws_sample.append(reviws)
                    reviws_stars.append(int(starts['content']))

In [16]:
%%time
parse_link(items_coments_link,reviws_sample,reviws_stars, page_count)

0 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=1;sort=helpful/
<Response [200]>
1 start prosess page https://rozetka.com.ua/nokia_5_ds_blue/p16104620/comments/page=1;sort=helpful/
<Response [200]>
2 start prosess page https://rozetka.com.ua/viaan_v182_black/p23575076/comments/page=1;sort=helpful/
<Response [200]>
3 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=1;sort=helpful/
<Response [200]>
4 start prosess page https://rozetka.com.ua/nokia_216_dual_sim_grey/p11684285/comments/page=1;sort=helpful/
<Response [200]>
5 start prosess page https://rozetka.com.ua/xiaomi_redmi_note4x_3_32_gry_eu/p17479997/comments/page=1;sort=helpful/
<Response [200]>
6 start prosess page https://rozetka.com.ua/nokia_3310_2017_ds_yellow/p14652296/comments/page=1;sort=helpful/
<Response [200]>
7 start prosess page https://rozetka.com.ua/lenovo_vibe_p1_pro_silver/p9651793/comments/page=1;sort=helpful/
<Response [200]>


In [17]:
print(len(reviws_sample))
print(len(reviws_stars))
print(len(page_count))
print()
evaluate_stars_count(reviws_stars)

362
362
32

5 357
4 1
3 1
2 2
1 1
0 0


In [18]:
def get_deeper_page_link(page_count,items_coments_link) :
    items_coments_link_deep_page = []
    for indx,page_c in enumerate(page_count) :
        if (page_c > 1) :
            link = items_coments_link[indx]

            for p in range(1,page_c + 1) :
                items_coments_link_deep_page.append(link.split('page=')[0] + 'page=' + str(p) + ';sort=helpful/')
    return items_coments_link_deep_page

In [19]:
items_coments_link_deep_page = get_deeper_page_link(page_count,items_coments_link)
len(items_coments_link_deep_page)

938

In [20]:
%%time
parse_link(items_coments_link_deep_page,reviws_sample,reviws_stars, page_count)

0 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=1;sort=helpful/
<Response [200]>
1 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=2;sort=helpful/
<Response [200]>
2 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=3;sort=helpful/
<Response [200]>
3 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=4;sort=helpful/
<Response [200]>
4 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=5;sort=helpful/
<Response [200]>
5 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=6;sort=helpful/
<Response [200]>
6 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=7;sort=helpful/
<Response [200]>
7 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=8;sort=helpful/
<

<Response [200]>
64 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=65;sort=helpful/
<Response [200]>
65 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=66;sort=helpful/
<Response [200]>
66 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=67;sort=helpful/
<Response [200]>
67 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=68;sort=helpful/
<Response [200]>
68 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=69;sort=helpful/
<Response [200]>
69 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=70;sort=helpful/
<Response [200]>
70 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=71;sort=helpful/
<Response [200]>
71 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722

<Response [200]>
128 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=129;sort=helpful/
<Response [200]>
129 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=130;sort=helpful/
<Response [200]>
130 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=131;sort=helpful/
<Response [200]>
131 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=132;sort=helpful/
<Response [200]>
132 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=133;sort=helpful/
<Response [200]>
133 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=134;sort=helpful/
<Response [200]>
134 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds_black/p3818722/comments/page=135;sort=helpful/
<Response [200]>
135 start prosess page https://rozetka.com.ua/samsung_galaxy_j7_ds

<Response [200]>
193 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=32;sort=helpful/
<Response [200]>
194 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=33;sort=helpful/
<Response [200]>
195 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=34;sort=helpful/
<Response [200]>
196 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=35;sort=helpful/
<Response [200]>
197 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=36;sort=helpful/
<Response [200]>
198 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=37;sort=helpful/
<Response [200]>
199 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=38;sort=helpful/
<Response [200]>
200 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/

<Response [200]>
257 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=96;sort=helpful/
<Response [200]>
258 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=97;sort=helpful/
<Response [200]>
259 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=98;sort=helpful/
<Response [200]>
260 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=99;sort=helpful/
<Response [200]>
261 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=100;sort=helpful/
<Response [200]>
262 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=101;sort=helpful/
<Response [200]>
263 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_black/p3818036/comments/page=102;sort=helpful/
<Response [200]>
264 start prosess page https://rozetka.com.ua/samsung_galaxy_j5_ds_bla

...

<Response [200]>
636 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=15;sort=helpful/
<Response [200]>
637 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=16;sort=helpful/
<Response [200]>
638 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=17;sort=helpful/
<Response [200]>
639 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=18;sort=helpful/
<Response [200]>
640 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=19;sort=helpful/
<Response [200]>
641 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=20;sort=helpful/
<Response [200]>
642 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=21;sort=helpful/
<Response [200]>
643 start prosess page https://rozetka.com.ua/nokia_6_ds_silver/p14653682/comments/page=22;sort=helpful/
<Response [200]>
644 sta

<Response [200]>
702 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=1;sort=helpful/
<Response [200]>
703 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=2;sort=helpful/
<Response [200]>
704 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=3;sort=helpful/
<Response [200]>
705 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=4;sort=helpful/
<Response [200]>
706 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=5;sort=helpful/
<Response [200]>
707 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=6;sort=helpful/
<Response [200]>
708 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=7;sort=helpful/
<Response [200]>
709 start prosess page https://rozetka.com.ua/huawei_y6_pro_wh/p7239508/comments/page=8;sort=helpful/
<Response [200]>
710 start prosess page https://

<Response [200]>
770 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=11;sort=helpful/
<Response [200]>
771 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=12;sort=helpful/
<Response [200]>
772 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=13;sort=helpful/
<Response [200]>
773 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=14;sort=helpful/
<Response [200]>
774 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=15;sort=helpful/
<Response [200]>
775 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=16;sort=helpful/
<Response [200]>
776 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=17;sort=helpful/
<Response [200]>
777 start prosess page https://rozetka.com.ua/samsung_sm_j510hzkdsek/p9142067/comments/page=18;sort=h

<Response [200]>
835 start prosess page https://rozetka.com.ua/apple_iphone_6s_32gb_gold/p12136244/comments/page=8;sort=helpful/
<Response [200]>
836 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=1;sort=helpful/
<Response [200]>
837 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=2;sort=helpful/
<Response [200]>
838 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=3;sort=helpful/
<Response [200]>
839 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=4;sort=helpful/
<Response [200]>
840 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=5;sort=helpful/
<Response [200]>
841 start prosess page https://rozetka.com.ua/apple_iphone_5s_16gb_space_gray_me432rk_a/p6800444/comments/page=6;sort=helpful/
<Respon

<Response [200]>
893 start prosess page https://rozetka.com.ua/sony_xperia_x_dual_f5122_rose_gold/p24251962/comments/page=16;sort=helpful/
<Response [200]>
894 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=1;sort=helpful/
<Response [200]>
895 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=2;sort=helpful/
<Response [200]>
896 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=3;sort=helpful/
<Response [200]>
897 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=4;sort=helpful/
<Response [200]>
898 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=5;sort=helpful/
<Response [200]>
899 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=6;sort=helpful/
<Response [200]>
900 start prosess page https://rozetka.com.ua/nokia_3_ds_silver/p14653556/comments/page=7;sort=helpful/
<Response [200

In [21]:
print(len(reviws_sample))
print(len(reviws_stars))
print(len(page_count))
print()
evaluate_stars_count(reviws_stars)

4199
4199
970

5 2959
4 883
3 205
2 87
1 65
0 0


##### Save results

In [22]:
reviws_sample = list(map(str.lower,reviws_sample))

df1 = pd.DataFrame()
df1['review'] = reviws_sample
df1['starts'] = reviws_stars
df1['review'] = [r.replace('\n','') for r in df1['review']]
df1.head()

,review,starts
0,при выборе телефона samsung galaxy j7 читал мн...,5
1,"писать много не буду, все уже сказано до меня....",5
2,"по себе знаю, как норм отзыв может помочь в вы...",5
3,"я так скажу,как человек меняющий аппараты част...",5
4,поражают коменты в которых пишут что слабая ка...,5


### Cheapest products

    There are a lot of good reviews, but little bad reviews. Because product sorted by rating.
    Let's get cheapest products.

In [24]:
items_coments_link = get_items_link('https://rozetka.com.ua/mobile-phones/c80003/sort=cheap/')
print(len(items_coments_link))

32


In [25]:
reviws_sample = []
reviws_stars = []
page_count = []

In [26]:
%%time
parse_link(items_coments_link,reviws_sample,reviws_stars, page_count)

0 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=1;sort=helpful/
<Response [200]>
1 start prosess page https://rozetka.com.ua/viaan_v182_black/p23575076/comments/page=1;sort=helpful/
<Response [200]>
2 start prosess page https://rozetka.com.ua/prestigio_pfp1180duoblack/p9244015/comments/page=1;sort=helpful/
<Response [200]>
3 start prosess page https://rozetka.com.ua/fly_ff179/p11019355/comments/page=1;sort=helpful/
<Response [200]>
4 start prosess page https://rozetka.com.ua/astro_a177_red/p9961900/comments/page=1;sort=helpful/
<Response [200]>
5 start prosess page https://rozetka.com.ua/viaan_v1820_black/p24520909/comments/page=1;sort=helpful/
<Response [200]>
6 start prosess page https://rozetka.com.ua/nomi_i184bk_gr/p10681766/comments/page=1;sort=helpful/
<Response [200]>
7 start prosess page https://rozetka.com.ua/prestigio_pfp1240duoblack/p9244344/comments/page=1;sort=helpful/
<Response [200]>
8 start prosess page https://rozetka.com.ua/viaan_v24

In [27]:
print(len(reviws_sample))
print(len(reviws_stars))
print(len(page_count))
print()
evaluate_stars_count(reviws_stars)

228
228
30

5 172
4 32
3 6
2 11
1 7
0 0


In [28]:
items_coments_link_deep_page = get_deeper_page_link(page_count,items_coments_link)
len(items_coments_link_deep_page)

194

In [29]:
%%time
parse_link(items_coments_link_deep_page,reviws_sample,reviws_stars, page_count)

0 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=1;sort=helpful/
<Response [200]>
1 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=2;sort=helpful/
<Response [200]>
2 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=3;sort=helpful/
<Response [200]>
3 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=4;sort=helpful/
<Response [200]>
4 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=5;sort=helpful/
<Response [200]>
5 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=6;sort=helpful/
<Response [200]>
6 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=7;sort=helpful/
<Response [200]>
7 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/comments/page=8;sort=helpful/
<Response [200]>
8 start prosess page https://rozetka.com.ua/globex_mint_red/p6956558/com

<Response [200]>
70 start prosess page https://rozetka.com.ua/nomi_i177_metal_grey/p7110429/comments/page=5;sort=helpful/
<Response [200]>
71 start prosess page https://rozetka.com.ua/nomi_i177_metal_grey/p7110429/comments/page=6;sort=helpful/
<Response [200]>
72 start prosess page https://rozetka.com.ua/nomi_i177_metal_grey/p7110429/comments/page=7;sort=helpful/
<Response [200]>
73 start prosess page https://rozetka.com.ua/nomi_i177_metal_grey/p7110429/comments/page=8;sort=helpful/
<Response [200]>
74 start prosess page https://rozetka.com.ua/nomi_i177_metal_grey/p7110429/comments/page=9;sort=helpful/
<Response [200]>
75 start prosess page https://rozetka.com.ua/astro_a178_black/p11551089/comments/page=1;sort=helpful/
<Response [200]>
76 start prosess page https://rozetka.com.ua/astro_a178_black/p11551089/comments/page=2;sort=helpful/
<Response [200]>
77 start prosess page https://rozetka.com.ua/astro_a178_black/p11551089/comments/page=3;sort=helpful/
<Response [200]>
78 start prosess

<Response [200]>
137 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=6;sort=helpful/
<Response [200]>
138 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=7;sort=helpful/
<Response [200]>
139 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=8;sort=helpful/
<Response [200]>
140 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=9;sort=helpful/
<Response [200]>
141 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=10;sort=helpful/
<Response [200]>
142 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=11;sort=helpful/
<Response [200]>
143 start prosess page https://rozetka.com.ua/sigma_mobile_comfort_50_mini3_bg/p2042292/comments/page=12;sort=helpful/
<Response [200]>
144 start prosess page https://roze

In [30]:
print(len(reviws_sample))
print(len(reviws_stars))
print(len(page_count))
print()
evaluate_stars_count(reviws_stars)

840
840
217

5 428
4 218
3 65
2 66
1 63
0 0


##### Save result

In [31]:
reviws_sample = list(map(str.lower,reviws_sample))

df2 = pd.DataFrame()
df2['review'] = reviws_sample
df2['starts'] = reviws_stars

df2['review'] = [r.replace('\n','') for r in df2['review']]
df2.head()

,review,starts
0,покупал маме и честно говоря это первый телефо...,5
1,отличнейший телефон учитывая его цену. покупал...,5
2,хороший телефон достоинства: свежая система io...,5
3,телефон прийшов за 2 дні після замовлення. хор...,5
4,"в подарунок дали стартовий , дрібниця але приє...",5


In [38]:
df = pd.DataFrame()
df['review'] = list(df2.review.values) + list(df1.review.values)
df['starts'] = list(df2.starts.values) + list(df1.starts.values)

df['target'] = np.where(df.starts.values >= 4,1,0)
df.head()

,review,starts,target
0,покупал маме и честно говоря это первый телефо...,5,1
1,отличнейший телефон учитывая его цену. покупал...,5,1
2,хороший телефон достоинства: свежая система io...,5,1
3,телефон прийшов за 2 дні після замовлення. хор...,5,1
4,"в подарунок дали стартовий , дрібниця але приє...",5,1


In [39]:
df.to_csv('reviews.csv',encoding='utf-8',index=False)